In [1]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt

def draw_boxplot_from_json():
    """
    Đọc file JSON kết quả và vẽ lại box plot
    Input: File JSON từ các thư mục nodes_150, nodes_200, ...
    Output: Biểu đồ box plot lưu vào thư mục draw_output
    """
    
    # ĐƯỜNG DẪN
    base_input_dir = "D:/Year 4/tiến hóa/project/kaggle/output/draw_hoitu"
    output_dir = "D:\Year 4\\tiến hóa\project\kaggle\output\draw_hoitu\draw_output"
    
    # Tạo thư mục output
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\n{'='*80}")
    print("VẼ LẠI BOX PLOT TỪ FILE JSON")
    print(f"{'='*80}\n")
    
    # Danh sách số nodes
    node_counts = [150, 200, 250, 300, 350, 400, 450, 550]
    
    # Duyệt qua từng thư mục
    for N in node_counts:
        folder_name = f"nodes_{N}"
        json_file = os.path.join(base_input_dir, folder_name, f"results_{folder_name}.json")
        
        # Kiểm tra file tồn tại
        if not os.path.exists(json_file):
            print(f"⚠️ Bỏ qua: {json_file} không tồn tại")
            continue
        
        print(f"\n{'='*60}")
        print(f"Xử lý: {folder_name}")
        print(f"{'='*60}")
        
        # Đọc file JSON
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            print(f"✅ Đọc file: {json_file}")
        except Exception as e:
            print(f"❌ Lỗi đọc file: {e}")
            continue
        
        # Lấy dữ liệu results
        results = data.get('results', {})
        
        if len(results) == 0:
            print(f"⚠️ Không có dữ liệu trong file")
            continue
        
        # ============================================
        # VẼ BOX PLOT
        # ============================================
        fig, ax = plt.subplots(figsize=(20, 8))
        
        # Tên các file (trục X)
        file_names = sorted(results.keys())  # ['nodes_150_1', 'nodes_150_2', ...]
        x_positions = np.arange(len(file_names))
        
        # Cấu hình cho 4 thuật toán
        algorithms = ['GA', 'PSO', 'PSOver2', 'PSOver3']
        colors = ['#FF6B6B', '#4ECDC4', '#FFD93D', '#95E1D3']
        markers = ['o', 'o', 'o', 'o']  # Hình dạng marker khác nhau
        linestyles = ['-', '--', '-.', ':']  # Kiểu đường thẳng khác nhau
        
        # Độ rộng offset cho mỗi thuật toán
        offset_width = 0.15
        offsets = [-1.5 * offset_width, -0.5 * offset_width, 
                   0.5 * offset_width, 1.5 * offset_width]
        
        # Vẽ cho mỗi thuật toán
        for alg_idx, (alg, color, marker, linestyle, offset) in enumerate(
            zip(algorithms, colors, markers, linestyles, offsets)
        ):
            for file_idx, filename in enumerate(file_names):
                # Kiểm tra thuật toán có trong file này không
                if alg not in results[filename]:
                    continue
                
                alg_data = results[filename][alg]
                values = alg_data.get('values', [])
                
                if len(values) == 0:
                    continue
                
                x_pos = x_positions[file_idx] + offset
                
                # Vẽ đường thẳng dọc từ min đến max
                min_val = min(values)
                max_val = max(values)
                ax.plot([x_pos, x_pos], [min_val, max_val], 
                       color=color, linestyle=linestyle, linewidth=2, alpha=0.7)
                
                # Vẽ các điểm dữ liệu
                ax.scatter([x_pos] * len(values), values, 
                          color=color, marker=marker, s=10, 
                          edgecolors='black', linewidths=1, 
                          alpha=0.8, zorder=3)
                
                # Vẽ median (đường ngang)
                average_val = np.mean(values)
                ax.plot([x_pos - 0.03, x_pos + 0.03], [average_val, average_val], 
                       color='black', linewidth=3, zorder=4)
        
        # Tạo legend
        legend_elements = [
            plt.Line2D([0], [0], color=color, marker=marker, linestyle=linestyle,
                      markersize=8, linewidth=2, label=alg)
            for alg, color, marker, linestyle in zip(algorithms, colors, markers, linestyles)
        ]
        ax.legend(handles=legend_elements, loc='upper right', fontsize=12, 
                 framealpha=0.9, edgecolor='black')
        
        # Cấu hình trục
        ax.set_xticks(x_positions)
        ax.set_xticklabels(file_names, rotation=45, ha='right', fontsize=11)
        ax.set_xlabel('Dataset Files', fontweight='bold', fontsize=14)
        ax.set_ylabel('Travel Time (s)', fontweight='bold', fontsize=14)
        ax.set_title(f'Routing Algorithm Comparison - {folder_name}', 
                    fontweight='bold', fontsize=16)
        
        
        plt.tight_layout()
        
        # Lưu biểu đồ
        chart_file = os.path.join(output_dir, f'boxplot_{folder_name}.png')
        plt.savefig(chart_file, dpi=200, bbox_inches='tight')
        plt.close()
        
        print(f"✅ Biểu đồ đã lưu: {chart_file}")
        
        # ============================================
        # IN BẢNG THỐNG KÊ
        # ============================================
        print(f"\n{'='*100}")
        print(f"BẢNG THỐNG KÊ - {folder_name}")
        print(f"{'='*100}")
        print(f"{'File':<18} {'Algorithm':<12} {'Mean':<10} {'Median':<10} {'Std':<10} {'Min':<10} {'Max':<10}")
        print("-" * 100)
        
        for filename in file_names:
            for alg in algorithms:
                if alg in results[filename]:
                    stats = results[filename][alg]
                    print(f"{filename:<18} {alg:<12} {stats['mean']:<10.2f} "
                          f"{stats['median']:<10.2f} {stats['std']:<10.2f} "
                          f"{stats['min']:<10.2f} {stats['max']:<10.2f}")
            print("-" * 100)
        
        print(f"\n✅ Hoàn thành {folder_name}!\n")
    
    print(f"\n{'='*80}")
    print(f"✅ HOÀN THÀNH! Tất cả biểu đồ đã lưu tại: {output_dir}")
    print(f"{'='*80}\n")




In [2]:
## sửa
import json
import os
import numpy as np
import matplotlib.pyplot as plt

def draw_boxplot_from_json():
    """
    Đọc file JSON kết quả và vẽ lại box plot
    Input: File JSON từ các thư mục nodes_150, nodes_200, ...
    Output: Biểu đồ box plot lưu vào thư mục draw_output
    """
    
    # ĐƯỜNG DẪN
    base_input_dir = "D:\\Year 4\\tiến hóa\\draw_hoitu"
    output_dir = "D:\\Year 4\\tiến hóa\\draw_hoitu\\draw_output"
    
    # Tạo thư mục output
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\n{'='*80}")
    print("VẼ LẠI BOX PLOT TỪ FILE JSON")
    print(f"{'='*80}\n")
    
    # Danh sách số nodes
    node_counts = [150, 200, 250, 300, 350, 400, 450, 550]
    
    # Duyệt qua từng thư mục
    for N in node_counts:
        folder_name = f"nodes_{N}"
        json_file = os.path.join(base_input_dir, folder_name, f"results_{folder_name}.json")
        
        # Kiểm tra file tồn tại
        if not os.path.exists(json_file):
            print(f"⚠️ Bỏ qua: {json_file} không tồn tại")
            continue
        
        print(f"\n{'='*60}")
        print(f"Xử lý: {folder_name}")
        print(f"{'='*60}")
        
        # Đọc file JSON
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            print(f"✅ Đọc file: {json_file}")
        except Exception as e:
            print(f"❌ Lỗi đọc file: {e}")
            continue
        
        # Lấy dữ liệu results
        results = data.get('results', {})
        
        if len(results) == 0:
            print(f"⚠️ Không có dữ liệu trong file")
            continue
        
        # ============================================
        # VẼ BOX PLOT
        # ============================================
        fig, ax = plt.subplots(figsize=(20, 8))
        
        # Tên các file (trục X)
        file_names = sorted(results.keys())  # ['nodes_150_1', 'nodes_150_2', ...]
        x_positions = np.arange(len(file_names))
        
        # Cấu hình cho 4 thuật toán
        algorithms = ['GA', 'PSO', 'PSOver2', 'PSOver3']
        colors = ['#FF6B6B', '#4ECDC4', '#FFD93D', '#95E1D3']
        markers = ['o', '^', 's', 'd']  # Hình dạng marker khác nhau
        linestyles = ['-', '--', '-.', ':']  # Kiểu đường thẳng khác nhau
        
        # Độ rộng offset cho mỗi thuật toán
        offset_width = 0.15
        offsets = [-1.5 * offset_width, -0.5 * offset_width, 
                   0.5 * offset_width, 1.5 * offset_width]
        
        # ✅ Biến lưu giá trị max để tính toán giới hạn trục Y
        all_values = []
        
        # Vẽ cho mỗi thuật toán
        for alg_idx, (alg, color, marker, linestyle, offset) in enumerate(
            zip(algorithms, colors, markers, linestyles, offsets)
        ):
            for file_idx, filename in enumerate(file_names):
                # Kiểm tra thuật toán có trong file này không
                if alg not in results[filename]:
                    continue
                
                alg_data = results[filename][alg]
                values = alg_data.get('values', [])
                
                if len(values) == 0:
                    continue
                
                # ✅ Thu thập tất cả giá trị để tính max
                all_values.extend(values)
                
                x_pos = x_positions[file_idx] + offset
                
                # Vẽ đường thẳng dọc từ min đến max
                min_val = min(values)
                max_val = max(values)
                ax.plot([x_pos, x_pos], [min_val, max_val], 
                       color=color, linestyle=linestyle, linewidth=2, alpha=0.7)
                
                # Vẽ các điểm dữ liệu
                ax.scatter([x_pos] * len(values), values, 
                          color=color, marker=marker, s=20, 
                          edgecolors='black', linewidths=1, 
                          alpha=0.8, zorder=3)
                
                # Vẽ median (đường ngang)
                average_val = np.mean(values)
                ax.plot([x_pos - 0.03, x_pos + 0.03], [average_val, average_val], 
                       color='black', linewidth=1, zorder=4)
        
        # ✅ Tăng giới hạn trục Y thêm 10%
        if len(all_values) > 0:
            y_min = min(all_values)
            y_max = max(all_values)
            y_range = y_max - y_min
            ax.set_ylim(y_min - 0.05 * y_range, y_max + 0.20 * y_range)  # Thêm 10% phía trên
        
        # Tạo legend với tên thuật toán mới
        legend_labels = {
            'GA': 'GA',
            'PSO': 'PSO',
            'PSOver2': 'PSO_Adaptive Noise',
            'PSOver3': 'PSO_Lévy Flight Noise'
        }

        legend_elements = [
            plt.Line2D([0], [0], color=color, marker=marker, linestyle=linestyle,
                    markersize=8, linewidth=2, label=legend_labels[alg])  # ✅ Dùng tên mới
            for alg, color, marker, linestyle in zip(algorithms, colors, markers, linestyles)
        ]
        ax.legend(handles=legend_elements, loc='upper right', fontsize=12, 
                framealpha=0.95, edgecolor='black')
        # ✅ Cấu hình trục
        ax.set_xticks(x_positions)
        ax.set_xticklabels(file_names, rotation=0, ha='center', fontsize=11) 
        #ax.set_xlabel('Dataset Files', fontweight='bold', fontsize=14)
        ax.set_ylabel('Travel Time (s)', fontweight='bold', fontsize=14)
        ax.set_title(f'Routing Algorithm Comparison - {folder_name}', 
                    fontweight='bold', fontsize=16)
        
        
        plt.tight_layout()
        
        # Lưu biểu đồ
        chart_file = os.path.join(output_dir, f'boxplot_{folder_name}.png')
        plt.savefig(chart_file, dpi=200, bbox_inches='tight')
        plt.close()
        
        print(f"✅ Biểu đồ đã lưu: {chart_file}")
        
        # ============================================
        # IN BẢNG THỐNG KÊ
        # ============================================
        print(f"\n{'='*100}")
        print(f"BẢNG THỐNG KÊ - {folder_name}")
        print(f"{'='*100}")
        print(f"{'File':<18} {'Algorithm':<12} {'Mean':<10} {'Median':<10} {'Std':<10} {'Min':<10} {'Max':<10}")
        print("-" * 100)
        
        for filename in file_names:
            for alg in algorithms:
                if alg in results[filename]:
                    stats = results[filename][alg]
                    print(f"{filename:<18} {alg:<12} {stats['mean']:<10.2f} "
                          f"{stats['median']:<10.2f} {stats['std']:<10.2f} "
                          f"{stats['min']:<10.2f} {stats['max']:<10.2f}")
            print("-" * 100)
        
        print(f"\n✅ Hoàn thành {folder_name}!\n")
    
    print(f"\n{'='*80}")
    print(f"✅ HOÀN THÀNH! Tất cả biểu đồ đã lưu tại: {output_dir}")
    print(f"{'='*80}\n")


# ============================================
# CHẠY HÀM
# ============================================
if __name__ == "__main__":
    draw_boxplot_from_json()


VẼ LẠI BOX PLOT TỪ FILE JSON

⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_150\results_nodes_150.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_200\results_nodes_200.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_250\results_nodes_250.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_300\results_nodes_300.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_350\results_nodes_350.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_400\results_nodes_400.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_450\results_nodes_450.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_550\results_nodes_550.json không tồn tại

✅ HOÀN THÀNH! Tất cả biểu đồ đã lưu tại: D:\Year 4\tiến hóa\draw_hoitu\draw_output



In [3]:
if __name__ == "__main__":
    draw_boxplot_from_json()



VẼ LẠI BOX PLOT TỪ FILE JSON

⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_150\results_nodes_150.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_200\results_nodes_200.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_250\results_nodes_250.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_300\results_nodes_300.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_350\results_nodes_350.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_400\results_nodes_400.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_450\results_nodes_450.json không tồn tại
⚠️ Bỏ qua: D:\Year 4\tiến hóa\draw_hoitu\nodes_550\results_nodes_550.json không tồn tại

✅ HOÀN THÀNH! Tất cả biểu đồ đã lưu tại: D:\Year 4\tiến hóa\draw_hoitu\draw_output

